In [ ]:
import os
import glob

import csv
import json

import numbers
import statistics
import random
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img

from datetime import datetime 

import ipyleaflet
import ipywidgets as widgets
from functools import partial
import ipyevents

## Cruise Data Functions

In [ ]:
def get_track_coords(adcp_map):
    """
    get lat/lon points of track for active cruise by reading /[cruise]/[instrument]/cruise_pts_fnames.csv
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :return: pd.DataFrame of lat, lon, and filename
    """
    cruise = adcp_map.state['cruise'] # get active cruise
    return(pd.read_csv(get_cr_instr_fldr(cruise,adcp_map.state['instr'])+'cruise_pts_fnames.csv'))

In [ ]:
def make_antpath(adcp_map):
    """
    create antpath of active cruise track from lat/lon points of cruise track
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :return: ipyleaflet.AntPath object - antpath of cruise track
    """
    cruise = adcp_map.state['cruise']
    
    track_pts = get_track_coords(adcp_map)
    track_pts = track_pts.drop(columns=['File_Prefix'])
    track_pts = np.array(track_pts).tolist()
    
    ant_path = ipyleaflet.AntPath(
        locations=[
            track_pts
        ],
        dash_array=[1, 10],
        delay=3000,
        color='blue',
        pulse_color='white'
    )
    return ant_path

In [ ]:
def read_geoJSON_track(adcp_map): 
    """
    read in and return pre-saved geoJSON feature collection of lines (as LineString features) representing the path of the ship during each ADCP file
    pre-saved geoJSON file is at /[cruise_code]/[instr_code]/geoJSON_track.geojson (see README)
    each LineString feature has the following properties:
    -'name' (string) - the ID of the file, but just the date and number, not the instrument name, eg. "2004_365_28800"
    -'activated' (boolean, default False) - whether that line segment is selected in the GUI
    -'gottows' (boolean) - whether there was a tow performed in that section of the cruise path
    -'idx' (int) - index of that cruise path section 
    -'tidxs' (list of ints, default []) - list of indices of tows that were performed at least partially in that section of the cruise path (
    -'teidxs' (list of ints, default []) - list of indices of tows that ENDED in that segment of the cruise path (regardless of whether they started in that segment)
    -'bath_exists' (boolean) - is there a bathymetry file for this ADCP file?
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :return: geoJSON feature collection with one LineString feature for each ADCP file
    """
    cruise = adcp_map.state['cruise']
    geofile = get_cr_instr_fldr(cruise,adcp_map.state['instr'])+'geoJSON_track.geojson'
    with open(geofile) as f:
        geojson = json.load(f)
    return(geojson)

In [ ]:
def make_track_layer(adcp_map):
    """
    create thin black line layer along cruise track
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :return: ipyleaflet.GeoJSON object of thin black line segments along cruise track
    """
    cruise = adcp_map.state['cruise']
    geo = read_geoJSON_track(adcp_map)
    style={'fillOpacity': 1, 'weight':1,'color':'black'}
    geo = ipyleaflet.GeoJSON(
        data=geo, 
        style=style,
        name='track'
    )
    return(geo)

In [ ]:
def make_track_marker_layer(adcp_map):
    """
    create line segment layer along cruise track - these will be the track segments that can be selected to show ADCP data
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :return: ipyleaflet.GeoJSON object of line segments along cruise track
    """
    
    cruise = adcp_map.state['cruise']
    triangles = read_geoJSON_track(adcp_map)
    style={'fillOpacity': 0.2}
    geo = ipyleaflet.GeoJSON(
            data=triangles, 
            style = style,
            style_callback = set_color1,
            name='track_markers'
    )
    
    # initialize properties
    geo.activated = -1 # idx of selected track segment --> default -1, i.e., nothing selected
    geo.style_callback_str = 1 # strange workaround to force refresh of layer - see comments for set_color1()
    idxs = [f['properties']['idx'] for f in geo.data['features']] # idxs in sequential order, with the track segment at the start of the cruise having idx=0
    geo.max_idx = max(idxs)
    
    return (geo)

## Map Layer Functions

In [ ]:
def clear_map(adcp_map):
    """
    remove all layers from the map except the base layer
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    for i in range(1, len(adcp_map.layers)): # this would be more efficient if it removed from bottom up rather than top down - could change if slow
        adcp_map.remove_layer(adcp_map.layers[1])

In [ ]:
def clear_geoJSON(adcp_map):
    """
    remove all geoJSON layers 
    (removes the geoJSON cruise tracks)
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    for ll in adcp_map.layers: 
        if type(ll) == ipyleaflet.leaflet.GeoJSON:
            adcp_map.remove_layer(ll)

In [ ]:
def get_track_marker_layer(adcp_map): 
    """
    get the geoJSON track_marker_layer from the map
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :return: ipyleaflet.geoJSON track marker layer
    """
    for l in adcp_map.layers:
        if l.name == 'track_markers':
            return(l)

## Event Functions

In [ ]:
def activate_cruise_segment(adcp_map,idx):
    """
    activate segment idx: set geo.activated = idx and force a refresh of the layer to update colors 
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :param feature: the int indicating the index of the active track segment 
    """
    geo = get_track_marker_layer(adcp_map) # get the geoJSON cruise path layer
    if geo.activated != idx: # if idx is not already activated:
        geo.data['features'][idx]['properties']['activated'] = True
        geo.data['features'][geo.activated]['properties']['activated'] = False
        geo.activated = idx # store activated idx as a property of the layer
        
        # odd workaround - I could not force the layer to refresh without changing its style, so I used two identical style attributes and just switch between them to get a refresh 
        if geo.style_callback_str == 1:
            geo.style_callback = set_color2 
            geo.style_callback_str = 2
        else:
            geo.style_callback = set_color1
            geo.style_callback_str = 1
        
        ff = geo.data['features'][idx]
        update_adcp_fig(adcp_map,ff) # update figure

In [ ]:
def update_tow_html(adcp_map,feature):
    """
    update the tow HTML label to display the organisms present in any tow performed in the active track segment
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :param feature: the active geoJSON track segment for which data should be displayed
    """
    geo = get_track_marker_layer(adcp_map)
    tow_species = {'EsuperbaNum':'E. superba','SalpAggNum':'S. thompsoni','ThysanoeNum':'T. mac','EcrystalNum':'E. crystal.'}
    tidxs = feature["properties"]["tidxs"]
    tows = adcp_map.state['tows']
    s = ""
    for t in tidxs:
        if s == "":
            #s = '<b>Tow creatures</b><font size="-2"><br>(mL/1000 m3)<br></font>' # if in volume
            s = '<b>Organisms in Tow (Counts)</b><br>' # if in number of indivs
            s = s + "Tow # " + str(tows['TowID'][t]) + "<br>"
        else:
            s = "<br>"
        for j, td in enumerate(tow_species):
            n = tows[td][t]
            if isinstance(n, numbers.Number):
                if n == -1:
                    n = 'Not Counted'
                else:
                    n = str(round(n))
            s = s+tow_species[td]+": "+n+"<br>"
        
    adcp_map.state['tow_html'].value = s

In [ ]:
def update_adcp_fig(adcp_map,feature):
    """
    updates the ADCP data figure and associated HTML labels based on newly selected feature (feature)
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    :param feature: the active geoJSON track segment for which data should be displayed
    """
    cruise = adcp_map.state['cruise']
    fname = feature["properties"]['name']
    
    adcp_map.state['fileprefix_html'].value = 'File: ' + feature["properties"]['name'] # update fileprefix_html
        
    f1 = '_nb150_SvRel-Run1-figAnnotated.png' # check for this figure first - this one has tows and CTD casts annotated on it
    f2 = '_nb150_SvRel-Run1-fig.png' # back-up fig if f1 doesn't exist
    prefx = get_fig_fldr()+cruise+'/'+adcp_map.state['instr']+'/lg'+fname
    file_name1 = prefx+f1
    file_name2 = prefx+f2

    if os.path.exists(file_name1):
        file = open(file_name1, "rb")
        update_adcp_image = file.read()
        adcp_map.state['adcp_image'].value = update_adcp_image
    elif os.path.exists(file_name2):
        file = open(file_name2, "rb")
        update_adcp_image = file.read()
        adcp_map.state['adcp_image'].value = update_adcp_image
    
    else: # no ADCP data for this track segment
        #default image
        file = open(get_meta_fldr()+"no-data.png", "rb")
        default_image = file.read()
        adcp_map.state['adcp_image'].value=default_image

    adcp_map.state['time_html'].value = feature['properties']['timestamp_html'] # update time_html
    
    update_tow_html(adcp_map,feature) # update tow_html

In [ ]:
def set_color1(feature):
    """
    function to set style (selected vs. not selected) of the track segments 
    note odd workaround: set_color1 and set_color2 are the same function, repeated because for some reason I cannot get it to update the layer's style unless I change the style function

    :param feature: track segment geoJSON feature
    """
    prop = 'gottows'
    if feature['properties'][prop]: # tows
        if feature['properties']['activated']:
            return {
                'color': 'magenta',
                'fillColor': 'magenta',
                'weight': 10,
                'opacity':0.8
            }
        else:
            return {
                'color': 'pink',
                'fillColor': 'pink',
                'weight': 5,
                'opacity':0.7
            }      
    else: # no tows
        if feature['properties']['activated']:
            return {
                'color': 'yellow',
                'fillColor': 'yellow',
                'weight': 10,
                'opacity':0.8
            }     
        else:
            return {
                'color': 'blue',
                'fillColor': 'blue',
                'weight': 5,
                'opacity':0
            }

def set_color2(feature):
    """
    function to set style (selected vs. not selected) of the track segments 
    note odd workaround: set_color1 and set_color2 are the same function, repeated because for some reason I cannot get it to update the layer's style unless I change the style function

    :param feature: track segment geoJSON feature
    """
    prop = 'gottows'
    if feature['properties'][prop]: # tows
        if feature['properties']['activated']:
            return {
                'color': 'magenta',
                'fillColor': 'magenta',
                'weight': 10,
                'opacity':0.8
            }
        else:
            return {
                'color': 'pink',
                'fillColor': 'pink',
                'weight': 5,
                'opacity':0.7
            }      
    else: # no tows
        if feature['properties']['activated']:
            return {
                'color': 'yellow',
                'fillColor': 'yellow',
                'weight': 10,
                'opacity':0.8
            }     
        else:
            return {
                'color': 'blue',
                'fillColor': 'blue',
                'weight': 5,
                'opacity':0
            }

## MAP

In [ ]:
def add_cruise_dropdown(adcp_map):
    """
    add a dropdown menu to select cruise by year 
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    crs = get_cruises()
    crs.sort()
    cr_yrs = [year_from_cruise(x) for x in crs]
    dropdown = widgets.Dropdown(
            options=cr_yrs,
            value=cr_yrs[0],
            description="Cruise:",
    )

    def on_click(change):
        clear_geoJSON(adcp_map)   
        cr_yr = change["new"]
        cruise = pal_cruise_from_year(cr_yr)
        adcp_map.state['cruise'] = cruise
        add_cruise_data(adcp_map)


    dropdown.observe(on_click, "value")
    dropdown = ipyleaflet.WidgetControl(widget=dropdown, position="bottomleft")

    adcp_map.add_control(dropdown)

In [ ]:
def add_adcp_image(adcp_map):
    """
    add the ADCP data figure to the map
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    
    #default image
    file = open(get_meta_fldr()+"no-data.png", "rb")
    default_image = file.read()

    if adcp_map.state['screensize'] == "large":
        imagewidth = 1100 #1200 x 1000 was too big
        imageheight = 1000 
    elif adcp_map.state['screensize'] == "small":
        imagewidth = 800 #1200 x 1000 was too big
        imageheight = 600 

    adcp_image = widgets.Image(
            value=default_image,
            format='png',
            width=imagewidth,
            height=imageheight
    )

    control_adcp_image = ipyleaflet.WidgetControl(widget=adcp_image, position="topright")
    adcp_map.add_control(control_adcp_image)

    adcp_map.state['adcp_image'] = adcp_image

In [ ]:
def add_html_labels(adcp_map):
    """
    add the html labels for time, file prefix, and tow info to the map
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    
    # time html label
    time_html = widgets.HTML("Date and Time")
    time_html.layout.margin = "0px 20px 0px 20px"
    control_time_html = ipyleaflet.WidgetControl(widget=time_html, position="topleft")
    adcp_map.add_control(control_time_html)
    adcp_map.state['time_html'] = time_html

    # fileprefix html label
    fileprefix_html = widgets.HTML("File Prefix")
    fileprefix_html.layout.margin = "0px 20px 0px 20px"
    control_fileprefix_html = ipyleaflet.WidgetControl(widget=fileprefix_html, position="topleft")
    adcp_map.add_control(control_fileprefix_html)
    adcp_map.state['fileprefix_html'] = fileprefix_html


    #tow html label
    tow_html = widgets.HTML("Tow Data")
    tow_html.layout.margin = "0px 20px 0px 20px"
    control_tow_html = ipyleaflet.WidgetControl(widget=tow_html, position="bottomright")
    adcp_map.add_control(control_tow_html)
    adcp_map.state['tow_html'] = tow_html

In [ ]:
def add_cruise_data(adcp_map):
    """
    add the data from the active cruise to the map
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    
    if not 'antpath' in adcp_map.state: # antpath doesn't already exist - add it
        antpath = make_antpath(adcp_map)
        adcp_map.add(antpath)
        adcp_map.state['antpath'] = antpath
    else: # antpath already exists - change the locations
        # there seems to be a bug in ipyleaflet where the map does not refresh correctly when you add a new antpath object, so I'm avoiding that by just using one antpath object and changing location
        adcp_map.state['antpath'].locations = make_antpath(adcp_map).locations
    
    geo = make_track_layer(adcp_map)
    adcp_map.add_control(geo)
    
    cruise = adcp_map.state['cruise']
    geo = make_track_marker_layer(adcp_map)
    
    def click_cruise_segment(feature,**kwargs):
        """
        callback function for cruise segment lines' click listener

        activates the clicked segment
        """
        idx = feature['properties']['idx']
        activate_cruise_segment(adcp_map,idx)

    geo.on_click(click_cruise_segment)
    
    #get idx to activate to start 
    try: # if there's a "map_viz_start_idx.csv" file...
        df = pd.read_csv(get_cr_instr_fldr(adcp_map.state['cruise'],adcp_map.state['instr'])+'map_viz_start_idx.csv')
        start_idx = df[df['Cruise'] == adcp_map.state['cruise']]['Start_Idx'][0]
    except: # if there isn't, just set start_idx=0
        start_idx = 0
        
    adcp_map.add_control(geo)
    activate_cruise_segment(adcp_map,start_idx)

In [ ]:
def add_adcp_size_button(adcp_map):
    """
    add a checkbox to show/hide the ADCP backscatter image
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    adcp_size_button = widgets.ToggleButton(
        value=False,
        description='Enlarge ADCP image',
        disabled=False,
        button_style='', 
        tooltip='Make the ADCP figure larger'
    )
    adcp_size_button_control = ipyleaflet.WidgetControl(widget=adcp_size_button,position='bottomleft')
    adcp_map.add_control(adcp_size_button_control)
    def on_adcp_size_button_changed(b):
        if adcp_size_button.value: # large 
            imagewidth = 1100 #1200 x 1000 was too big
            imageheight = 1000 
            adcp_map.state['adcp_image'].width = imagewidth
            adcp_map.state['adcp_image'].height = imageheight
        else:
            imagewidth = 800 #1200 x 1000 was too big
            imageheight = 600 

            adcp_map.state['adcp_image'].width = imagewidth
            adcp_map.state['adcp_image'].height = imageheight
            
    adcp_size_button.observe(on_adcp_size_button_changed)
    
    adcp_map.state['adcp_size_button'] = adcp_size_button


In [ ]:
def add_show_adcp_checkbox(adcp_map):
    """
    add a checkbox to show/hide the ADCP backscatter image
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    show_adcp_checkbox = widgets.Checkbox(
        value=True,
        description='Show ADCP backscatter data',
        disabled=False,
        indent=False
    )
    show_adcp_checkbox.layout.justify_content = 'center'
    show_adcp_checkbox_control = ipyleaflet.WidgetControl(widget=show_adcp_checkbox,position='bottomleft')
    adcp_map.add_control(show_adcp_checkbox_control)
    def on_show_adcp_checkbox_changed(b):
        if show_adcp_checkbox.value:
            adcp_map.state['adcp_image'].layout.display = "block"
            adcp_map.state['adcp_size_button'].disabled = False # enable the adcp_size_button when the figure is shown
        else:
            adcp_map.state['adcp_image'].layout.display = "none"
            adcp_map.state['adcp_size_button'].disabled = True # disable the adcp_size_button when the figure is hidden
    show_adcp_checkbox.observe(on_show_adcp_checkbox_changed)


In [ ]:
def add_keypress_listeners(adcp_map):
    """
    add keypress listeners to allow the right/left arrowkeys to advance/back up the selected track segment
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    d = ipyevents.Event(source=adcp_map, watched_events=['keydown'])

    def advance_marker(): # move the selected marker forward by one
        geo = get_track_marker_layer(adcp_map)
        idx = geo.activated+1
        if idx <= geo.max_idx:
            activate_cruise_segment(adcp_map,idx)

    def backup_marker(): # move the selected marker backwards by one
        geo = get_track_marker_layer(adcp_map)
        idx = geo.activated-1
        if idx >= 0:
            activate_cruise_segment(adcp_map,idx)


    def handle_event(event):
        geo = get_track_marker_layer(adcp_map)
        for k,v in event.items():
            if k == "key":
                if v == "ArrowRight":
                    advance_marker()
                elif v == "ArrowLeft":
                    backup_marker()
    d.on_dom_event(handle_event)

In [ ]:
def add_tow_data(adcp_map):
    """
    read tow data as pd.DataFrame and add it to the map's state dict as adcp_map.state['tows']
    
    :param adcp_map: the ipyleaflet.Map object that forms the base of the visualization
    """
    tows = pd.read_csv(get_meta_fldr()+'tows_in_adcp_nb150.csv')

    # remove columns with invalid tow file - if tows occur outside of ADCP data recording period, tow files were recorded as nan
    tows.ADCP_StartFile = [str(x) for x in tows.ADCP_StartFile] # convert ADCP_StartFile column to strings
    tows = tows[tows.ADCP_StartFile != 'nan'] # remove any where the startfile is 'nan'
    tows = tows.reset_index(drop=True)
    
    tows.ADCP_EndFile = [str(x) for x in tows.ADCP_EndFile] # repeat with ADCP_EndFile
    tows = tows[tows.ADCP_EndFile != 'nan']
    tows = tows.reset_index(drop=True)
    
    # get just the file prefix (e.g., '2005_004_14400') for both start and end files - this will allow matching with correct figures later
    tmp = [str(x)[2:(x).rfind('_')] for x in tows["ADCP_StartFile"]] 
    tmp = [str(x)[0:str(x).rfind('_')] for x in tmp]
    tows['file_name_start'] = tmp
    tmp = [str(x)[2:(x).rfind('_')] for x in tows["ADCP_EndFile"]]
    tmp = [str(x)[0:str(x).rfind('_')] for x in tmp]
    tows['file_name_end'] = tmp
    
    adcp_map.state['tows'] = tows # add the pd.DataFrame to the adcp_map.state dict

In [ ]:
def adcp_map():
    """
    main function to create the ipyleaflet.Map visualization
    
    :return: ipyleaflet.Map object that forms the visualization 
    """
    adcp_map = get_base_map()
    adcp_map.state = {'screensize':'small','cruise':'lg0501','instr':'nb150'} # initialize dict to hold GUI state
    add_tow_data(adcp_map)
    add_cruise_dropdown(adcp_map)
    add_adcp_image(adcp_map)
    add_html_labels(adcp_map)
    add_cruise_data(adcp_map)
    add_adcp_size_button(adcp_map)
    add_show_adcp_checkbox(adcp_map)
    add_keypress_listeners(adcp_map)
    return(adcp_map)